In [105]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import requests
import json
import time
import re
from random import uniform

In [99]:
dirr = 'C:/Users/shic/Desktop/shinhan_simon/google_drive/crawler_data/'
dir_for_seunghun = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'

In [92]:
header = {
    'Accept' : 'application/json, text/javascript, */*; q=0.01',
    'Accept-Encoding' : 'gzip, deflate',
    'Accept-Language' : 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection' : 'keep-alive',
    'Content-Type' : 'application/json; charset=UTF-8',
    'Host' : 'app1.reb.or.kr:8082',
    'Referer' : 'http://app1.reb.or.kr:8082/stat004.html',
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
}

In [93]:
code_df = pd.read_csv(dirr+'법정동코드 전체자료.txt',encoding='CP949',sep='\t',
                      dtype={'법정동코드':'str',
                             '법정동명'  :'str',
                             '폐지여부'  :'str'})
code_df = code_df[['법정동코드','법정동명']]

In [94]:
code_df['cut'] = code_df['법정동코드'].str.slice(start=5)
code_df = code_df.loc[code_df['cut']=='00000',['법정동코드','법정동명']].reset_index(drop=True)
code_df['params'] = code_df['법정동코드'].str.slice(stop=5)
code_df

,법정동코드,법정동명,params
0,1100000000,서울특별시,11000
1,1111000000,서울특별시 종로구,11110
2,1114000000,서울특별시 중구,11140
3,1117000000,서울특별시 용산구,11170
4,1120000000,서울특별시 성동구,11200
...,...,...,...
456,4971000000,제주도 북제주군,49710
457,4972000000,제주도 남제주군,49720
458,5000000000,제주특별자치도,50000
459,5011000000,제주특별자치도 제주시,50110


In [ ]:
df_list = []
urll = 'http://app1.reb.or.kr:8082/APPSTASTICS004/001.json?provinceCode={}&period=10'
driver = webdriver.Chrome(dir_for_seunghun)

for roww in code_df.iterrows():
    url_now = urll.format(roww[1]['params'])
    driver.get(url_now)
    
    req_text = re.sub('<.*?>','',driver.page_source)
    
    req_json = json.loads(req_text)
    tmp_df = pd.DataFrame(req_json['list'])
    tmp_df['address'] = roww[1]['법정동명']
    
    time.sleep(uniform(0,2))
    
    df_list.append(tmp_df)